In [2]:
import pandas as pd
from datasets import Dataset
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          TrainingArguments, 
                          Trainer,
                          DataCollatorWithPadding)
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from nltk.corpus import stopwords as nltk_stopwords

# Загрузка размеченного датасета

In [3]:
# 1. Загружаем данные
df = pd.read_csv('geo-reviews-dataset-2023.csv') # Колонки: 'review', 'label'
df_1 = df.loc[df['label'] == 1]
df_2 = df.loc[df['label'] == 2]
df_3 = df.loc[df['label'] == 3]
df_4 = df.loc[df['label'] == 4]
df_5 = df.loc[df['label'] == 5]

df_new = pd.concat([df_1[0:1000], df_2[0:1000], df_3[0:1000], df_4[0:1000], df_5[0:1000]], ignore_index=True)
df_new = df_new.sample(frac=1).reset_index(drop=True) #перемешать
#df = df.sort_values(by='label')
#df = df[260:4600]
df_new.shape

(5000, 2)

In [4]:
df_new.head()

,review,label
0,"Вожу в Детский центр ""Планета ребёнок"" своего ...",5
1,"Я обожаю грузинскую кухню, но была разочарован...",2
2,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",5
3,"Доски как доски. Акустика правда лютая. А так,...",3
4,"Очень красивый вид и очень атмосферное место, ...",3


In [5]:
test_df = pd.concat([df_1[1001:1200], df_2[1001:1200], df_3[1001:1200], df_4[1001:1200], df_5[1001:1200]], ignore_index=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

# Предобработка отзывов

In [6]:
# Оставим в тексте только кириллические символы
def clear_text(text):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
    return " ".join(clear_text.split())

# напишем функцию удаляющую стоп-слова
def clean_stop_words(text, stopwords):
    text = [word for word in text.split() if word not in stopwords]
    return " ".join(text)

# загрузим список стоп-слов
stopwords = set(nltk_stopwords.words('russian'))
np.array(stopwords)

array({'они', 'так', 'мы', 'чтоб', 'при', 'об', 'почти', 'через', 'тебя', 'после', 'вы', 'эти', 'много', 'конечно', 'разве', 'более', 'даже', 'надо', 'наконец', 'всегда', 'и', 'чем', 'этот', 'тем', 'один', 'нельзя', 'ну', 'кто', 'быть', 'хоть', 'а', 'то', 'вдруг', 'три', 'были', 'во', 'куда', 'теперь', 'уже', 'ей', 'под', 'потому', 'перед', 'между', 'такой', 'раз', 'вам', 'бы', 'них', 'тут', 'его', 'ты', 'чуть', 'там', 'свою', 'того', 'их', 'уж', 'если', 'можно', 'потом', 'была', 'совсем', 'еще', 'чего', 'будто', 'тогда', 'тоже', 'со', 'с', 'когда', 'она', 'где', 'к', 'вас', 'какой', 'этой', 'опять', 'мой', 'всех', 'но', 'ее', 'нибудь', 'всю', 'ему', 'может', 'два', 'здесь', 'меня', 'из', 'ничего', 'или', 'себе', 'сейчас', 'я', 'эту', 'сам', 'нее', 'моя', 'лучше', 'другой', 'будет', 'же', 'него', 'ж', 'вот', 'было', 'как', 'том', 'впрочем', 'ведь', 'этого', 'по', 'для', 'до', 'зачем', 'о', 'над', 'больше', 'нет', 'ни', 'про', 'хорошо', 'чтобы', 'без', 'иногда', 'тот', 'какая', 'ним', '

In [7]:
import re
# Протестируем работу функции очистки текста
review = df_new['review'][np.random.randint(df_new.shape[0])]
print(review)
print('=======================================')
print(clean_stop_words((clear_text(review)), stopwords))

Отличная химчистка. Почистили зимний пуховик - как новенький стал! Опережают сроки оказания услуг. Удобная система скидок на последующие заказы. Хорошая упаковка (как полиэтиленовый кофр), уловные плечики. Однозначно рекомендую!  Не знаю как в других пунктах приема этой химчистки, но в ТЦ «Европа-СИТИ» все замечательно 👍 
отличная химчистка почистили зимний пуховик новенький стал опережают сроки оказания услуг удобная система скидок последующие заказы хорошая упаковка полиэтиленовый кофр уловные плечики однозначно рекомендую знаю других пунктах приема химчистки тц европа сити замечательно


In [8]:
import time
# Очистим тексты отзывов
start_clean = time.time()

df_new['review'] = df_new['review']\
     .apply(lambda x: clean_stop_words(clear_text(str(x)), stopwords))

print('Обработка текстов заняла: '+str(round(time.time() - start_clean, 2))+' секунд')

Обработка текстов заняла: 0.16 секунд


In [9]:
dataset = Dataset.from_pandas(df_new)

# 2. Загружаем токенизатор
model_name = 'sberbank-ai/ruBert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples['review'], truncation=True, padding=True, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 4. Разделяем на train/test
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)

# 5. Загружаем модель с нужным количеством классов
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# 6. Определяем метрики для оценки
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

In [11]:
# 7. Настраиваем гиперпараметры обучения
training_args = TrainingArguments(
    output_dir='./results_m2',          # Куда сохранять результаты
    eval_strategy="epoch",     # Оценивать после каждой эпохи
    learning_rate=2e-5,              # Скорость обучения
    per_device_train_batch_size=16,  # Размер батча
    per_device_eval_batch_size=16,
    num_train_epochs=3,              # Количество эпох
    weight_decay=0.01,               # Регуляризация
    save_strategy="epoch",
    logging_dir='./logs',
)

In [12]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [13]:
# 8. Создаем Data Collator для динамического дополнения последовательностей
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 9. Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 10. Запускаем обучение!
trainer.train()

# 11. Сохраняем модель
trainer.save_model('./my_sentiment_model_2')

C:\Users\andre\AppData\Local\Temp\ipykernel_13484\899701250.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.070456,0.552000,0.547044
2,1.069200,1.041454,0.550000,0.545054
3,1.069200,1.096782,0.548000,0.543654


In [14]:
import torch

In [15]:
model = AutoModelForSequenceClassification.from_pretrained('./my_sentiment_model_2')
tokenizer = AutoTokenizer.from_pretrained('./my_sentiment_model_2')
# Функция для предсказания настроения
def predict_sentiment(text):
    # Токенизируем текст и подготавливаем для модели
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    
    # Подаем на вход модели без вычисления градиентов (только инференс)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Получаем предсказания
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    # Названия классов, которые возвращает модель (уточните в документации к модели)
    classes = ['НЕГАТИВНЫЙ', 'ПОЗИТИВНЫЙ'] 
    
    # Находим индекс класса с максимальной вероятностью
    predicted_class_idx = predicted.argmax().item()
    sentiment = predicted_class_idx #classes[predicted_class_idx]
    confidence = predicted[0][predicted_class_idx].item()
    
    return sentiment, confidence

In [ ]:
# Пример
# Очистим тексты отзывов
test_df['review'] = test_df['review']\
     .apply(lambda x: clean_stop_words(clear_text(str(x)), stopwords))

In [41]:
# 1 минута на 1000 отзывов
test_df['prediction'] = test_df['review']\
        .apply(lambda x: predict_sentiment(str(x))[0])
test_df.head()

,review,label,prediction,spread_distance
0,обслуживание нормальное еда шведском столе ста...,3,4,1
1,тёплый просторный номер фен шампунь гель душа ...,3,4,1
2,великолепное место летом приехать палатками кр...,5,5,0
3,тихое местечко недалеко центратгорода кухня ми...,3,4,1
4,овощи битые помидоры гнилые честное мерзлый ка...,1,1,0


In [42]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(test_df['label'], test_df['prediction'])
print(matrix)

[[139  36  19   4   1]
 [ 48  68  56  25   2]
 [ 22  44  82  46   5]
 [  2  10  42 109  36]
 [  2   1   4  26 166]]


In [45]:
test_df['spread_distance'] = ((test_df['label'] - test_df['prediction']) ** 2)
test_df.head()

,review,label,prediction,spread_distance
0,обслуживание нормальное еда шведском столе ста...,3,4,1
1,тёплый просторный номер фен шампунь гель душа ...,3,4,1
2,великолепное место летом приехать палатками кр...,5,5,0
3,тихое местечко недалеко центратгорода кухня ми...,3,4,1
4,овощи битые помидоры гнилые честное мерзлый ка...,1,1,0


In [49]:
dups = test_df.pivot_table(index = ['spread_distance'], aggfunc ='size')
print(dups)

spread_distance
0     564
1     334
4      85
9       9
16      3
dtype: int64
